In [ ]:
from refchecker import LLMExtractor, LLMChecker
import json
import numpy as np


def extract(data_path1):
    extractor = LLMExtractor(
    claim_format='triplet', 
    model='bedrock/meta.llama3-1-70b-instruct-v1:0',
    batch_size=50,
    )
    data = json.load(open(data_path1))

    for d_id, d_all in enumerate(data):
        r_f1 = d_all['female1']
        r_f2 = d_all['female2']
        r_m = d_all['male1'] 
 
        response_extract_results1 = extractor.extract(batch_responses=r_f1)
        response_claims1 = [[c.content for c in res.claims] for res in response_extract_results1]        
        response_extract_results2 = extractor.extract(batch_responses=r_f2)
        response_claims2 = [[c.content for c in res.claims] for res in response_extract_results2]
        response_extract_results3 = extractor.extract(batch_responses=r_m)
        response_claims3 = [[c.content for c in res.claims] for res in response_extract_results3]
        claims = []
        claims.append(response_claims1)
        claims.append(response_claims2)
        claims.append(response_claims3)   

        d_all['claims'] = claims
                  
    
    with open("data/gender/claims.json", "w") as f:
        json.dump(data, f, indent=2)  
        
extract("data/age/age.json")


In [ ]:
from refchecker import LLMExtractor, LLMChecker
import json
import numpy as np
def check(data_path1):
    checker = LLMChecker(
        model = 'bedrock/meta.llama3-1-70b-instruct-v1:0', 
        batch_size=50,
    )
    
    data = json.load(open(data_path1))
    for index, item in enumerate(data):
        q = item['questions']
        c = item['claims']
        c_f1 = c[0]
        c_f2 = c[1]
        c_m = c[2]
        ref_f1 = item['female1']
        ref_f2 = item['female2']
        ref_m = item['male1']
        

        labels_f1_to_f2=checker.check(
            batch_claims= c_f1,   # 10 claim-list for female1 
            batch_questions= [q] * len(c_f1),  
            batch_references=[ref_f2] * len(c_f1),   # 10 response for female2
            is_joint=True,
            joint_check_num=10
            )
        labels_f2_to_f1=checker.check(
            batch_claims= c_f2,   # female2
            batch_questions=[q]*10,  
            batch_references=[ref_f1] * 10,   # female1
            is_joint=True,
            joint_check_num=10
            )   
        
        labels_f1_to_m=checker.check(
            batch_claims= c_f1,   # female1
            batch_questions=[q]*10,  
            batch_references=[ref_m] * 10,   # male1
            is_joint=True,
            joint_check_num=10
            )
        labels_m_to_f1=checker.check(   
            batch_claims= c_m,   # male1
            batch_questions=[q]*10,  
            batch_references=[ref_f1] * 10,   # female1
            is_joint=True,
            joint_check_num=10
            )   
        labels_f2_to_m=checker.check(
            batch_claims= c_f2,   # female2
            batch_questions=[q]*10,  
            batch_references=[ref_m] * 10,   # male1
            is_joint=True,
            joint_check_num=10
            )
        labels_m_to_f2=checker.check(
            batch_claims= c_m,   # male1
            batch_questions=[q]*10,  
            batch_references=[ref_f2] * 10,   # female2
            is_joint=True,
            joint_check_num=10
            )
        item['labels_f1_to_f2'] = labels_f1_to_f2
        item['labels_f2_to_f1'] = labels_f2_to_f1
        item['labels_f1_to_m'] = labels_f1_to_m
        item['labels_m_to_f1'] = labels_m_to_f1
        item['labels_f2_to_m'] = labels_f2_to_m
        item['labels_m_to_f2'] = labels_m_to_f2

    with open('data/age/labels.json', "w") as f:
        json.dump(data, f, indent=2)



check("data/age/claims.json")



In [1]:
from refchecker import LLMExtractor, LLMChecker
import json
import numpy as np
def check(data_path1):
    checker = LLMChecker(
        model = 'bedrock/meta.llama3-1-70b-instruct-v1:0', 
        batch_size=50,
    )
    
    data = json.load(open(data_path1))
    for index, item in enumerate(data):
        q = item['questions']
        c = item['claims']
        c_f1 = c[0]
        c_f2 = c[1]
        c_m = c[2]
        ref_f1 = item['female1']
        ref_f2 = item['female2']
        ref_m = item['male1']
        

        labels_f1_intra=checker.check(
            batch_claims= c_f1,   # 10 claim-list for female1 
            batch_questions= [q] * len(c_f1),  
            batch_references=[ref_f1] * len(c_f1),   # 10 response for female2
            is_joint=True,
            joint_check_num=10
            )
        
        labels_f2_intra=checker.check(
            batch_claims= c_f2,   # female2
            batch_questions=[q]*10,  
            batch_references=[ref_f2] * 10,   # female1
            is_joint=True,
            joint_check_num=10
            )
        
        item['labels_f1_intra'] = labels_f1_intra
        item['labels_f2_intra'] = labels_f2_intra

    with open('data/age/labels.json', "w") as f:
        json.dump(data, f, indent=2)



check("data/age/labels.json")


/opt/conda/envs/refchecker/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  0%|          | 0/5 [00:00<?, ?it/s]

litellm.RateLimitError: BedrockException - {"message":"Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again."} [sleep 10 seconds]


 40%|████      | 2/5 [00:17<00:22,  7.63s/it]

litellm.RateLimitError: BedrockException - {"message":"Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again."} [sleep 10 seconds]


...

 25%|██▌       | 1/4 [00:14<00:44, 14.67s/it]

litellm.RateLimitError: BedrockException - {"message":"Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again."} [sleep 10 seconds]


 75%|███████▌  | 3/4 [00:32<00:09,  9.17s/it]

litellm.RateLimitError: BedrockException - {"message":"Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again."} [sleep 10 seconds]


100%|██████████| 4/4 [00:45<00:00, 11.39s/it]
